In [2]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from os import listdir
import re

## List Semua Saham di Bursa Efek Indonesia

List tersebut diambil secara manual di [https://www.idx.co.id/data-pasar/data-saham/daftar-saham/](https://www.idx.co.id/data-pasar/data-saham/daftar-saham/). Kemudian datanya disimpan di dalam file `all_saham.csv`

In [19]:
all_saham = pd.read_csv('data/all_saham.csv')
all_saham

,kode,perusahaan,jumlah_saham,papan_pencatatan
0,AALI,Astra Agro Lestari Tbk.,1.924.688.333,UTAMA
1,ABBA,Mahaka Media Tbk.,2.755.125.000,Pengembangan
2,ABDA,Asuransi Bina Dana Arta Tbk.,620.806.680,PENGEMBANGAN
3,ABMM,ABM Investama Tbk.,2.753.165.000,Utama
4,ACES,Ace Hardware Indonesia Tbk.,17.150.000.000,Utama
...,...,...,...,...
761,YULE,Yulie Sekuritas Indonesia Tbk.,1.785.000.000,Pengembangan
762,ZBRA,Zebra Nusantara Tbk,2.510.706.263,Pengembangan
763,ZINC,Kapuas Prima Coal Tbk.,25.250.000.000,Pengembangan
764,ZONE,Mega Perintis Tbk.,870.171.478,PENGEMBANGAN


Nama saham masih banyak yang formatnya tidak sama sehingga akan dilakukan scraping nama perusahaan yang ada di website yahoo finance, sekaligus untuk melihat apakah data historynya ada atau tidak di yahoo finance

In [30]:
hasil = []
for index, saham in all_saham.iterrows():
    kode = saham['kode']
    
    try:
        URL = f"https://finance.yahoo.com/quote/{kode}.JK/profile?p={kode}.JK"
        page = requests.get(URL, headers={'user-agent': 'My app'})
        page = BeautifulSoup(page.content, 'html.parser')
        nama_perusahaan = page.find("h1", class_="D(ib) Fz(18px)").text
        nama_perusahaan = nama_perusahaan.split(" (")[0]    
    except:
        nama_perusahaan = None
    
    hasil.append({"kode" : kode, "nama_perusahaan" : nama_perusahaan})
    
hasil

[{'kode': 'AALI', 'nama_perusahaan': 'PT Astra Agro Lestari Tbk'},
 {'kode': 'ABBA', 'nama_perusahaan': 'PT Mahaka Media Tbk'},
 {'kode': 'ABDA', 'nama_perusahaan': 'PT Asuransi Bina Dana Arta Tbk'},
 {'kode': 'ABMM', 'nama_perusahaan': 'PT ABM Investama Tbk'},
 {'kode': 'ACES', 'nama_perusahaan': 'PT Ace Hardware Indonesia Tbk'},
 {'kode': 'ACST', 'nama_perusahaan': 'PT Acset Indonusa Tbk'},
 {'kode': 'ADES', 'nama_perusahaan': 'PT Akasha Wira International Tbk'},
 {'kode': 'ADHI', 'nama_perusahaan': 'PT Adhi Karya'},
 {'kode': 'ADMF', 'nama_perusahaan': 'PT Adira Dinamika Multi Finance Tbk'},
 {'kode': 'ADMG', 'nama_perusahaan': 'PT. Polychem Indonesia Tbk'},
 {'kode': 'ADRO', 'nama_perusahaan': 'PT Adaro Energy Tbk'},
 {'kode': 'AGAR', 'nama_perusahaan': 'PT Asia Sejahtera Mina Tbk'},
 {'kode': 'AGII', 'nama_perusahaan': 'PT Aneka Gas Industri Tbk'},
 {'kode': 'AGRO', 'nama_perusahaan': 'PT Bank Rakyat Indonesia Agroniaga Tbk'},
 {'kode': 'AGRS', 'nama_perusahaan': 'PT Bank IBK Indo

Buat `keyword` untuk setiap saham yang nantinya `keyword` tersebut akan dicocokkan dengan data berita. Jika `keyword` tersebut ada di dalam text berita maka berita tersebut milik saham dari `keyword` tersebut

In [ ]:
sahams = pd.DataFrame(hasil)

# keyword pertama adalah nama perusahaan yang dihilangkan tulisan PT dan TBK
sahams["keyword"] = sahams.nama_perusahaan.str.replace("PT ", "").str.replace(" Tbk", "").str.lower()
# sahams.to_csv("data/daftar_saham.csv")

Setelah itu dilakukan pengecekan ulang secara manual dan menambahkan beberapa `keyword` tambahan untuk beberapa saham tertentu yang kemudian datanya akan disimpan di `daftar_saham.csv`

In [13]:
sahams = pd.read_csv("data/daftar_saham.csv", index_col=0)
sahams

,kode,nama_perusahaan,keyword
0,AALI,PT Astra Agro Lestari Tbk,astra agro lestari
1,ABBA,PT Mahaka Media Tbk,mahaka media
2,ABDA,PT Asuransi Bina Dana Arta Tbk,asuransi bina dana arta
3,ABMM,PT ABM Investama Tbk,abm investama
4,ACES,PT Ace Hardware Indonesia Tbk,ace hardware indonesia
...,...,...,...
755,YULE,PT Yulie Sekuritas Indonesia Tbk,yulie sekuritas indonesia
756,ZBRA,PT Zebra Nusantara Tbk,zebra nusantara
757,ZINC,PT Kapuas Prima Coal Tbk,kapuas prima coal
758,ZONE,PT Mega Perintis Tbk,mega perintis


## Preprocessing Hasil Scraping

Yang dilakukan disini hanya membuang whitespace dan kata-kata yang selalu berulang disetiap berita, sebagai contoh di artikel dari bisnis.com selalu diawali dengan kata-kata `Bisnis.com, JAKARTA`

### Berita di Bisnis.com

#### Gabung Data
Gabung seluruh data yang ada di folder `data/isi_berita_bisnis/`

In [2]:
def gabung_data(path):
    files = listdir(path)

    data = None
    for file_txt in files:
        with open(f"{path}/{file_txt}", "r", encoding="utf-8") as f:
            isi = json.load(f)
            f.close()

        if data is None:
            data = pd.DataFrame(isi)
        else:
            data = pd.concat([data, pd.DataFrame(isi)])

    data = data.set_index("index").reset_index(drop=True)
    return data

path_isi_bisnis = "data/isi_berita_bisnis/"
data = gabung_data(path_isi_bisnis)

#### Hapus kata berulang diawal dan akhir artikel

Mengidentifikasi kata kata yang selalu berulang atau tidak ada maknanya yang berada diawal dan akhir dari berita. Proses ini dilakukan dengan cara manual, sehingga seluruh kata tersebut nantinya akan disimpan dalam file excel `stop_words_bisnis.xlsx`

In [ ]:
# Ambil 30 kata di depan dan dibelakang untuk dianalisis lebih lanjut
data['depan'] = data.isi2.str[0:30]
data['belakang'] = data.isi2.str[-30:]
data

Setelah diperhatikan untuk kata di awal artikel ada polanya, banyak artikel yang diawali dengan kota tempat ditulisnya berita tersebut dan terdapat pemisah antara kota tersebut dengan mulainya isi artikel, kurang lebih pemisahnya adalah `["—", "-", "--", "---", ":", "–"]`. Sebagai contoh `Bisnis.com, JAKARTA - Indonesia Fintech Societ...`. Oleh karena itu kita dapat memisahkannya dengan bantuan program karena memiliki pola tertentu

In [ ]:
def separate_tulisan_awal(text):
    if text is None:
        return None
    
    separator = ["—", "-", "--", "---", ":", "–"]
    for sep in separator:
        hasil = text.split(sep)
        if len(hasil) > 1:
            return hasil[0] + sep
    return None

hasil = data.depan.apply(separate_tulisan_awal)

(hasil
 .value_counts()
 .to_frame()
 .reset_index()
 .rename(columns={"index" : "text"})
 .sort_values("text")
 .to_excel("data/depan_bisnis.xlsx"))

hasil.value_counts()

Sementara itu untuk kalimat di akhir artikel tidak memiliki pola tertentu sehingga perlu dianalisis secara manual

In [ ]:
belakang = (data
            .belakang
            .value_counts()
            .to_frame()
            .reset_index()
            .rename(columns={"index" : "text", "belakang" : "jumlah"})
            .sort_values("jumlah", ascending=False))

belakang[belakang.jumlah > 10].to_excel("data/belakang_bisnis.xlsx")
belakang

#### Hapus Kata-Kata Tidak Perlu
Setelah dilakukan analisis secara manual yang sangat membosankan akhirnya semua list kata/kalimat yang tidak perlu berhasil disimpan di `data/stop_words_bisnis.xlsx`. Selanjutnya untuk setiap artikel akan dicek satu persatu. Hasilnya nanti akan disimpan dalam bentuk `json` karena kalau dalam `xlsx` terlalu kebesaran, hingga memunculkan warning

In [55]:
stop_words = pd.read_excel("data/stop_words_bisnis.xlsx").text.to_list()
stop_words

['Disclaimer: Berita ini tidak bertujuan mengajak membeli atau menjual saham. Keputusan investasi sepenuhnya ada di tangan pembaca. Bisnis.com tidak bertanggung jawab terhadap segala kerugian maupun keuntungan yang timbul dari keputusan investasi pembaca.',
 'SINGAPURA: ", "  >> BACA JUGA: * Ini dia REKSADANA baru ", "Untuk membaca berita-berita dan memperoleh referensi terpercaya dari Harian Bisnis Indonesia, silahkan klik epaper.bisnis.com.',
 'Saham apa saja yang menjadi\xa0top gainers\xa0dalam perdagangan Bursa Efek Indonesia hari ini? Berikut rinciannya: Sumber: BEI',
 'Anda juga bisa berlangganan epaper Bisnis Indonesia dengan register langsung ke Bisnis Indonesia edisi digital.',
 'Simak berita lainnya seputar topik_x000D_\n                                        artikel ini, di sini :',
 'mengemukakan ada 7 saham yang dapat dipertimbangkan pada perdagangan hari ini, yaitu:',
 'Kurs Transaksi Bank Indonesia (Rupiah) Sumber: Bank Indonesia',
 'Berikut lima saham yang mendapatkan 

In [59]:
def remove_word(text):
    if text is None:
        return text
    
    for sentance in stop_words:
        text = text.replace(sentance, "")

    # hapus whitespace diawal dan akhir artikel dan hapus semua line break
    text = text.strip().replace('\n', ' ').replace('\r', '')
    
    # hapus strip diawalan artikel
    text =  re.sub("^(-+)*?", "", text).strip()
    
    # jika awalannya seperti ini, hapus aja datanya, ga ada informasinya
    if text[:30] == "Simak berita lainnya seputar t":
        return None
    
    return text

data['isi'] = data.isi.apply(remove_word)
data

,judul,link,tanggal,is_sukses,isi
0,Krisis utang Eropa tekan harga saham,https://market.bisnis.com/read/20101201/190/25...,2010-12-01,True,
1,Nusantara Infrastructure bidik pendapatan Rp45...,https://market.bisnis.com/read/20101201/192/18...,2010-12-01,True,PT Nusantara Infrastructure Tbk (META) menarge...
2,Jasindo kembangkan bisnis syariah,https://market.bisnis.com/read/20101201/192/15...,2010-12-01,True,PT Asuransi Jasa Indonesia (Jasindo) berkomitm...
3,Harga minyak terkikis spekulasi dana talangan ...,https://market.bisnis.com/read/20101201/235/13...,2010-12-01,True,"Setelah minyak pemanas turun, minyak ikut terk..."
4,Harga emas naik dapat berkah dari krisis Eropa,https://market.bisnis.com/read/20101201/235/17...,2010-12-01,True,Kontrak emas menguat ke dua minggu tertinggi d...
...,...,...,...,...,...
181836,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",https://market.bisnis.com/read/20211209/192/14...,2021-12-09,True,Pengadilan Negeri Jakarta Pusat mengabulkan tu...
181837,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",https://market.bisnis.com/read/20211209/189/14...,2021-12-09,True,Kinerja indeks saham syariah Jakarta Islamic I...
181838,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",https://market.bisnis.com/read/20211209/189/14...,2021-12-09,True,Artha Sekuritas Indonesia memprediksi Indeks H...
181839,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",https://market.bisnis.com/read/20211209/192/14...,2021-12-09,True,Emiten pelayaran PT Pelita Samudera Shipping T...


In [60]:
depan = data.isi.str[:30]

In [61]:
counts = depan.value_counts()
counts[counts > 10]

                                  5688
Indeks harga saham gabungan (I    5206
Indeks Harga Saham Gabungan (I    3551
Otoritas Jasa Keuangan (OJK) m    1536
PT Bank Mandiri (Persero) Tbk.    1256
                                  ... 
Kenaikan suku bunga acuan Bank      11
Emiten perkebunan, PT Austindo      11
Pemerintah menggelar lelang su      11
Bursa Efek Indonesia telah men      11
PT Borneo Lumbung Energi & Met      11
Name: isi, Length: 1290, dtype: int64

In [62]:
counts.sort_index()[counts > 10].to_excel("test.xlsx")

In [63]:
data.to_json("data/berita_bisnis.json")

### Berita di Detik Finance

#### Gabung Data
Gabung semua data dari artikel detik yang sudah dikumpulkan yaitu yang ada di folder `data/isi_berita_detik/`

In [332]:
path_isi_detik = "data/isi_berita_detik/"
data = gabung_data(path_isi_detik)
data

,judul,link,tanggal,is_sukses,isi
0,OPEC Putuskan Kurangi Produksi,https://finance.detik.com/berita-ekonomi-bisni...,2004-04-01,True,\nDen Haag - Meskipun harga minyak saat ini me...
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",https://finance.detik.com/bursa-dan-valas/d-12...,2004-04-01,True,\nJakarta - Indeks Harga Saham Gabungan pada K...
2,Rupiah Dibuka Menguat di Rp 8570,https://finance.detik.com/moneter/d-122087/rup...,2004-04-01,True,"\nJakarta - Meskipun secara perlahan, nilai tu..."
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",https://finance.detik.com/moneter/d-122086/gre...,2004-04-01,True,\nJakarta - Dolar AS kembali jatuh terhadap Eu...
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,https://finance.detik.com/moneter/d-122084/dit...,2004-04-01,True,\nCibinong - Direktorat Jenderal (Ditjen) Paja...
...,...,...,...,...,...
354827,Penggantian Wesel KRL,https://finance.detik.com/foto-bisnis/d-489878...,2020-02-14,True,\nJakarta - PT KCI dan PT KAI Daerah Operasi 1...
354828,Berburu Aneka Sepeda Bekas di Jalan Malabar Ba...,https://finance.detik.com/foto-bisnis/d-489901...,2020-02-14,True,\nBandung - Jalan Malabar di Kota Bandung terk...
354829,Benang Gelasan Asli Sumedang Tembus Pasar Eksp...,https://finance.detik.com/foto-bisnis/d-489915...,2020-02-14,True,"\nJakarta - Sejak 1990-an, Kampung gelasan di ..."
354830,Bank Mantap Bagikan Deviden Rp 136 Miliar,https://finance.detik.com/foto-bisnis/d-489965...,2020-02-14,True,\nBali - Bank Mantap menggelar Rapat Umum Peme...


#### Clean Artikel
Sama dengan data pada `bisnis.com`, artikel paling sering diawali dengan nama tempat ketika artikel ditulis dan `-` menjadi pemisah antara tempat artikel ditulis dengan isi artikel. Kemudian diakhir artikel juga masih banyak kalimat yang tidak perlu seperti kalimat untuk melakukan promosi yang sama sekali tidak memiliki hubungan dengan apa yang dibahas pada artikel sehingga perlu dihapus. Untuk mengidentifikasi dilakukan secara manual dengan mengamati pola artikel secara umum

In [333]:
def clean_artikel(text):
    if text is None:
        return text
    
    # hapus line break
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ').strip()
    
    # identifikasi tempat penulisan artikel dan hapus tulisannya
    tmp = text.split("-")
    if (len(tmp) > 1) and (len(tmp[0]) < 27):
        awalan = tmp[0] + "-"
        text = text.replace(awalan, "").strip()
    
    # diakhir kalimat banyak diakhiri dengan identitas tertentu yang 
    # saya tidak mengerti juga apa maksudnya. Misalnnya (ads/afe)
    # polanya katanya selalu berada didalam kurung
    kalimat_akhir = text[-50:]
    try:
        to_remove = re.search("\(.+\)", kalimat_akhir)[0]
        text = text.replace(to_remove, "")
    except:
        pass
    
    # Menghapus kata-kata yang tidak perlu
    sentances = [
        "Lanjut ke halaman berikutnya >>>",
        "berikut ini:",
        "Berikut daftar lengkapnya",
        "[Gambas:Video 20detik]",
        "http://bit.ly/18anniversary.",
        "Berikut informasi selengkapnya.",
        "redaksi@detikFinance.com.",
        "Berikut informasi selengkapnya:",
        "http://bit.ly/promoakhirpekan.",
        "Semoga bermanfaat"
    ]
    
    for sentance in sentances:
        text = text.replace(sentance, "")
    
    # Menggunakan satu kalimat atau beberapa kalimat yang tidak perlu menggunakan regex
    # Setiap promosi/iklan biasa diawali dengan kata-kata tersebut
    # Oleh karena itu dengan bantuan regex akan dicari kata tersebut dan kemudian
    # seluruh kata/kalimat setelahnya akan dihapus
    # Sebagai contoh `re.sub('(Lihat juga Video: .+)$', "", text)` akan menghapus
    # seluruh kata/kalimat `Lihat juga Video` beserta kalimat-kalimat setelahnya
    
    text = re.sub('(Lihat juga Video: .+)$', "", text)
    text = re.sub('(Simak Video .+)$', "", text)
    text = re.sub('(Disclaimer: .+)$', "", text)
    text = re.sub('(Mau tahu informasi selengkapnya\? .+)$', "", text)
    text = re.sub('(Berikut berita selengkapnya .+)$', "", text)
    text = re.sub('(Untuk mengetahui informasi lebih lengkap, .+)$', "", text)
    text = re.sub('(Untuk informasi lebih lengkap, .+)$', "", text)
    text = re.sub('(Khusus Untuk 100 Orang Pembaca .+)$', "", text)
    text = re.sub('(Mau Menjadi Kaya Atau Bahkan Lebih Kaya .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo lain yang berlangsung .+)$', "", text)
    text = re.sub('(Untuk promo lebih lengkap, .+)$', "", text)
    text = re.sub('(Untuk melihat promo lainnya, .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo lainnya, .+)$', "", text)
    text = re.sub('(Untuk mengetahui promo .+)$', "", text)
    text = re.sub('(Dapatkan eBook Tentang "61 Rahasia .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda di Tahun 2018\? .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda di Tahun 2018 ini\? .+)$', "", text)
    text = re.sub('(Sudah Siapkah Anda Ditahun 2018\? .+)$', "", text)
    text = re.sub('(Saksikan juga video .+)$', "", text)
    text = re.sub('(Lihat Video: .+)$', "", text)
    text = re.sub('(Baca 5 berita .+)$', "", text)
    text = re.sub('(Bisa kirim ceritanya ke .+)$', "", text)
    text = re.sub('(Tonton video .+)$', "", text)
    text = re.sub('(Tonton juga Video: .+)$', "", text)    
    text = re.sub('(Untuk mengetahui informasi dari .+)$', "", text)
    text = re.sub('(Hasil Intisari Pemikiran Dan Praktek Saya .+)$', "", text)
    text = re.sub('(Dapatkan eBook Tentang .+)$', "", text)
    text = re.sub('(Mau Ikut Pelatihan dengan Materi Terbaru .+)$', "", text)
    text = re.sub('(Mau ikut diskusi soal Aturan Baru Taksi .+)$', "", text)
    text = re.sub('(Bagaimana Kaya Secara Aman di Tahun 2017 ke 2018 ini\? .+)$', "", text)
    text = re.sub('(Untuk mengetahui info lengkap .+)$', "", text)
    text = re.sub('(Mau eBook Tentang .+)$', "", text)
    text = re.sub('(Mau Menjadi Kaya Atau Bahkan Lebih Kaya\? .+)$', "", text)
    text = re.sub('(Ikuti terus berita tentang .+)$', "", text)
    text = re.sub('(Untuk melihat promo .+)$', "", text)
    text = re.sub('(GRATIS\. .+)$', "", text)
    text = re.sub('(Tonton juga .+)$', "", text)
    text = re.sub('(Hasil Intisari Pemikiran Dan Praktik Saya .+)$', "", text)
    text = re.sub('(Simak berita selengkapnya .+)$', "", text)
    text = re.sub('(Untuk mengetahui info .+)$', "", text)
    text = re.sub("(Siap 'Bantu Jokowi Cari Menteri'\? .+)$", "", text)
    text = re.sub("(bagi detikers yang .+)$", "", text)
    text = re.sub("(Ingin tahu kisah lain .+)$", "", text)
    text = re.sub("(Untuk ilmu yang lebih lengkap lagi, .+)$", "", text)
    text = re.sub("(Untuk mengetahui beragam promo .+)$", "", text)
    text = re.sub("(Simak ulasan lengkapnya .+)$", "", text)
    text = re.sub("(Di Jakarta dibuka workshop sehari tentang bagaimana .+)$", "", text)
    text = re.sub("(detikcom bersama BRI .+)$", "", text)
    text = re.sub("(detikcom bersama Bank BRI .+)$", "", text)
    text = re.sub("(Promo ini masih ditambah .+)$", "", text)
    text = re.sub("(Berbagai promo ini masih ditambah .+)$", "", text)
    text = re.sub("(Program ini untuk melengkapi promo .+)$", "", text)
    text = re.sub("(bagi yang bertransaksi .+)$", "", text)
    text = re.sub("(Untuk transaksi pembelian emas batangan .+)$", "", text)
    text = re.sub("(Untuk transaksi pembelian Emas .+)$", "", text)
    text = re.sub("(Dapatkan berbagai promo .+)$", "", text)
    text = re.sub("(Baca berita lainnya .+)$", "", text)
    
    return text.strip()

In [334]:
data['isi'] = data.isi.apply(clean_artikel)
data.to_json("data/berita_detik.json")
data

,judul,link,tanggal,is_sukses,isi
0,OPEC Putuskan Kurangi Produksi,https://finance.detik.com/berita-ekonomi-bisni...,2004-04-01,True,"Meskipun harga minyak saat ini melonjak, OPEC ..."
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",https://finance.detik.com/bursa-dan-valas/d-12...,2004-04-01,True,Indeks Harga Saham Gabungan pada Kamis (1/4/20...
2,Rupiah Dibuka Menguat di Rp 8570,https://finance.detik.com/moneter/d-122087/rup...,2004-04-01,True,"Meskipun secara perlahan, nilai tukar Rupiah p..."
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",https://finance.detik.com/moneter/d-122086/gre...,2004-04-01,True,Dolar AS kembali jatuh terhadap Euro setelah a...
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,https://finance.detik.com/moneter/d-122084/dit...,2004-04-01,True,Direktorat Jenderal (Ditjen) Pajak berhasil me...
...,...,...,...,...,...
354827,Penggantian Wesel KRL,https://finance.detik.com/foto-bisnis/d-489878...,2020-02-14,True,PT KCI dan PT KAI Daerah Operasi 1 Jakarta men...
354828,Berburu Aneka Sepeda Bekas di Jalan Malabar Ba...,https://finance.detik.com/foto-bisnis/d-489901...,2020-02-14,True,Jalan Malabar di Kota Bandung terkenal sebagai...
354829,Benang Gelasan Asli Sumedang Tembus Pasar Eksp...,https://finance.detik.com/foto-bisnis/d-489915...,2020-02-14,True,"Sejak 1990-an, Kampung gelasan di Tanjungsari,..."
354830,Bank Mantap Bagikan Deviden Rp 136 Miliar,https://finance.detik.com/foto-bisnis/d-489965...,2020-02-14,True,Bank Mantap menggelar Rapat Umum Pemegang Saha...


## Identifikasi Artikel yang Membahas Saham

### Satukan Data Isi dan Judul
Di file yang menyimpan seluruh judul ada variabel kategori yang tidak ada di data isi artikel. Oleh karena itu data tersebut akan digabung terlebih dahulu. Untuk menggabungkannya akan digunakan menggunakan variabel `link` karena variabel tersebut pasti unik untuk setiap artikel

#### Data Bisnis.com

In [335]:
bisnis_artikel = pd.read_json("data/berita_bisnis.json")
bisnis_artikel

,judul,link,tanggal,is_sukses,isi
0,Krisis utang Eropa tekan harga saham,https://market.bisnis.com/read/20101201/190/25...,2010-12-01,True,
1,Nusantara Infrastructure bidik pendapatan Rp45...,https://market.bisnis.com/read/20101201/192/18...,2010-12-01,True,PT Nusantara Infrastructure Tbk (META) menarge...
2,Jasindo kembangkan bisnis syariah,https://market.bisnis.com/read/20101201/192/15...,2010-12-01,True,PT Asuransi Jasa Indonesia (Jasindo) berkomitm...
3,Harga minyak terkikis spekulasi dana talangan ...,https://market.bisnis.com/read/20101201/235/13...,2010-12-01,True,"Setelah minyak pemanas turun, minyak ikut terk..."
4,Harga emas naik dapat berkah dari krisis Eropa,https://market.bisnis.com/read/20101201/235/17...,2010-12-01,True,Kontrak emas menguat ke dua minggu tertinggi d...
...,...,...,...,...,...
181836,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",https://market.bisnis.com/read/20211209/192/14...,2021-12-09,True,Pengadilan Negeri Jakarta Pusat mengabulkan tu...
181837,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",https://market.bisnis.com/read/20211209/189/14...,2021-12-09,True,Kinerja indeks saham syariah Jakarta Islamic I...
181838,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",https://market.bisnis.com/read/20211209/189/14...,2021-12-09,True,Artha Sekuritas Indonesia memprediksi Indeks H...
181839,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",https://market.bisnis.com/read/20211209/192/14...,2021-12-09,True,Emiten pelayaran PT Pelita Samudera Shipping T...


In [337]:
bisnis_judul = pd.read_csv("data/link_berita_bisnis.csv")
bisnis_judul

,judul,kategori,publish_date,link,tanggal
0,Krisis utang Eropa tekan harga saham,Saham & Sekuritas,2010-12-01 00:48:00,https://market.bisnis.com/read/20101201/190/25...,2010-12-01
1,Nusantara Infrastructure bidik pendapatan Rp45...,Korporasi,2010-12-01 01:11:00,https://market.bisnis.com/read/20101201/192/18...,2010-12-01
2,Jasindo kembangkan bisnis syariah,Korporasi,2010-12-01 01:32:00,https://market.bisnis.com/read/20101201/192/15...,2010-12-01
3,Harga minyak terkikis spekulasi dana talangan ...,Emas,2010-12-01 01:36:00,https://market.bisnis.com/read/20101201/235/13...,2010-12-01
4,Harga emas naik dapat berkah dari krisis Eropa,Emas,2010-12-01 01:39:00,https://market.bisnis.com/read/20101201/235/17...,2010-12-01
...,...,...,...,...,...
181836,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
181837,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181838,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181839,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09


In [448]:
ordered_colums = ["judul", "isi", "kategori", "publish_date", "link"]
rename = {"judul" : "title", "kategori" : "category", "isi" : "content", "publish_date" : "published_at"}
data_bisnis = (bisnis_artikel
               .set_index("link")
               .join(bisnis_judul[["kategori", "publish_date", "link"]].set_index("link"))
               .reset_index()
               .drop("is_sukses", axis=1)[ordered_colums]
               .rename(columns=rename))

data_bisnis['website'] = "bisnis.com"
data_bisnis

,title,content,category,published_at,link,website
0,PPN kegiatan usaha bank umum dipertegas,Direktorat Jenderal Pajak mempertegas perlakua...,Perbankan,2010-12-01 03:23:00,https://finansial.bisnis.com/read/20101201/90/...,bisnis.com
1,"BI: Kredit bank sepekan tumbuh Rp10,47 triliun",Bank Indonesia mencatat pertumbuhan kredit per...,Perbankan,2010-12-01 03:36:00,https://finansial.bisnis.com/read/20101201/90/...,bisnis.com
2,Parlemen incar penetapan dewan komisioner OJK,Pembahasan mengenai penentuan dewan komisioner...,Perbankan,2010-12-02 14:03:00,https://finansial.bisnis.com/read/20101202/90/...,bisnis.com
3,Mandiri tawarkan klausula arbitrase di perjanj...,PT Bank Mandiri Tbk menawarkan pencantuman kla...,Perbankan,2010-12-04 04:50:00,https://finansial.bisnis.com/read/20101204/90/...,bisnis.com
4,Ada anomali pada kredit perbankan,Kinerja kredit perbankan nasional mengalami an...,Perbankan,2010-12-05 13:32:00,https://finansial.bisnis.com/read/20101205/90/...,bisnis.com
...,...,...,...,...,...,...
209860,Rupiah Menguat Bersama Mata Uang Negara Asean,Nilai tukar rupiah dibuka menguat pada pagi in...,Kurs,2021-12-09 09:41:00,https://market.bisnis.com/read/20211209/93/147...,bisnis.com
209861,"Rupiah Ditutup Melemah, Padahal Mayoritas Mata...",Nilai tukar rupiah terhadap dolar AS ditutup m...,Kurs,2021-12-09 15:52:00,https://market.bisnis.com/read/20211209/93/147...,bisnis.com
209862,Kabar Soal Omicron dan Cadangan AS Kerek Harga...,Harga minyak mentah menguat tipis pada akhir p...,Komoditas,2021-12-09 06:31:00,https://market.bisnis.com/read/20211209/94/147...,bisnis.com
209863,Harga Aset Kripto Bitcoin Ethereum Masih Merah...,Harga sejumlah aset kripto dengan kapitalisasi...,Komoditas,2021-12-09 16:12:00,https://market.bisnis.com/read/20211209/94/147...,bisnis.com


In [449]:
data_bisnis.to_json("data/fix_bisnis_artikel.json")

#### Berita di Detik Finance

In [355]:
detik_artikel = pd.read_json("data/berita_detik.json")
detik_artikel

,judul,link,tanggal,is_sukses,isi
0,OPEC Putuskan Kurangi Produksi,https://finance.detik.com/berita-ekonomi-bisni...,2004-04-01,True,"Meskipun harga minyak saat ini melonjak, OPEC ..."
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",https://finance.detik.com/bursa-dan-valas/d-12...,2004-04-01,True,Indeks Harga Saham Gabungan pada Kamis (1/4/20...
2,Rupiah Dibuka Menguat di Rp 8570,https://finance.detik.com/moneter/d-122087/rup...,2004-04-01,True,"Meskipun secara perlahan, nilai tukar Rupiah p..."
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",https://finance.detik.com/moneter/d-122086/gre...,2004-04-01,True,Dolar AS kembali jatuh terhadap Euro setelah a...
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,https://finance.detik.com/moneter/d-122084/dit...,2004-04-01,True,Direktorat Jenderal (Ditjen) Pajak berhasil me...
...,...,...,...,...,...
354827,Penggantian Wesel KRL,https://finance.detik.com/foto-bisnis/d-489878...,2020-02-14,True,PT KCI dan PT KAI Daerah Operasi 1 Jakarta men...
354828,Berburu Aneka Sepeda Bekas di Jalan Malabar Ba...,https://finance.detik.com/foto-bisnis/d-489901...,2020-02-14,True,Jalan Malabar di Kota Bandung terkenal sebagai...
354829,Benang Gelasan Asli Sumedang Tembus Pasar Eksp...,https://finance.detik.com/foto-bisnis/d-489915...,2020-02-14,True,"Sejak 1990-an, Kampung gelasan di Tanjungsari,..."
354830,Bank Mantap Bagikan Deviden Rp 136 Miliar,https://finance.detik.com/foto-bisnis/d-489965...,2020-02-14,True,Bank Mantap menggelar Rapat Umum Pemegang Saha...


In [357]:
detik_judul = pd.read_csv("data/list_berita_detik.csv")
detik_judul

,judul,sub_judul,kategori,timestamp_publish,link,tanggal
0,OPEC Putuskan Kurangi Produksi,hasil sidang opec di wina,berita ekonomi bisnis,1080770940,https://finance.detik.com/berita-ekonomi-bisni...,01-04-2004
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",NaN,bursa dan valas,1080787800,https://finance.detik.com/bursa-dan-valas/d-12...,01-04-2004
2,Rupiah Dibuka Menguat di Rp 8570,NaN,moneter,1080788280,https://finance.detik.com/moneter/d-122087/rup...,01-04-2004
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",NaN,moneter,1080791460,https://finance.detik.com/moneter/d-122086/gre...,01-04-2004
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,NaN,moneter,1080799020,https://finance.detik.com/moneter/d-122084/dit...,01-04-2004
...,...,...,...,...,...,...
340627,"UMKM Resah Produk Impor, Ketua MPR Minta Ada P...",NaN,berita ekonomi bisnis,1638892559,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021
340628,Siap-siap Tarif PBB Bakal Naik,infografis,infografis,1638892835,https://finance.detik.com/infografis/d-5845257...,07-12-2021
340629,Korban Erupsi Semeru Diusulkan Dapat Keringana...,NaN,berita ekonomi bisnis,1638893242,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021
340630,"Segera Diresmikan Jokowi, Menhub Cek Bandara T...",NaN,infrastruktur,1638895340,https://finance.detik.com/infrastruktur/d-5845...,07-12-2021


Ternyata data judul dan data artikel memiliki jumlah data yang berbeda. Hal ini dikarenakan ada berita yang dilakukan scaping dua atau tiga kali karena ketika percobaan pertama artikelnya gagal di scraping. Oleh karena itu datanya yang gagal tersebut akan dihapus terlebih dahulu. Setelah itu datanya akan digabung menggunakan variabel `link`

In [425]:
link = detik_artikel.link.value_counts()
link = link[link > 1].index

data_double = detik_artikel[detik_artikel.link.isin(link)]
is_null = data_double[data_double.isi.isnull()]
is_null = is_null.drop_duplicates()
index_to_delete = is_null.index.to_list()
data_double = data_double.drop(index_to_delete)

link = data_double.link.value_counts()
link = link[link > 1].index
double = data_double[data_double.link.isin(link)]
index_to_delete = index_to_delete + double[double.is_sukses == False].index.to_list()
index_to_delete

[6156,
 6938,
 11031,
 13139,
 13925,
 16692,
 22835,
 26260,
 29372,
 30076,
 30259,
 30337,
 30722,
 32929,
 33472,
 34083,
 43725,
 44418,
 44423,
 44452,
 44467,
 44475,
 44520,
 44527,
 44538,
 44551,
 44642,
 44647,
 44703,
 44706,
 44713,
 44714,
 44721,
 44788,
 44802,
 44850,
 44909,
 44934,
 44968,
 44982,
 45007,
 45038,
 45046,
 45066,
 45103,
 45108,
 45166,
 45227,
 45247,
 45305,
 45322,
 45350,
 45367,
 45409,
 45411,
 45467,
 45524,
 45531,
 45592,
 45664,
 45681,
 45682,
 45723,
 45727,
 45780,
 45833,
 45847,
 45851,
 45913,
 45957,
 45976,
 46041,
 46046,
 46077,
 46109,
 46155,
 46166,
 46171,
 46186,
 46223,
 46227,
 46286,
 46339,
 46404,
 46412,
 46450,
 46472,
 46481,
 46533,
 46551,
 46593,
 46651,
 46703,
 46722,
 46772,
 46792,
 46849,
 46883,
 46894,
 46904,
 46944,
 47064,
 47091,
 47166,
 47187,
 47233,
 47289,
 47313,
 47358,
 47399,
 47409,
 47443,
 47468,
 47534,
 47564,
 47633,
 47684,
 47695,
 47797,
 47852,
 47869,
 47917,
 48008,
 48054,
 48055,
 4

In [426]:
detik_artikel = detik_artikel[~detik_artikel.index.isin(index_to_delete)]
detik_artikel

,judul,link,tanggal,is_sukses,isi
0,OPEC Putuskan Kurangi Produksi,https://finance.detik.com/berita-ekonomi-bisni...,2004-04-01,True,"Meskipun harga minyak saat ini melonjak, OPEC ..."
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",https://finance.detik.com/bursa-dan-valas/d-12...,2004-04-01,True,Indeks Harga Saham Gabungan pada Kamis (1/4/20...
2,Rupiah Dibuka Menguat di Rp 8570,https://finance.detik.com/moneter/d-122087/rup...,2004-04-01,True,"Meskipun secara perlahan, nilai tukar Rupiah p..."
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",https://finance.detik.com/moneter/d-122086/gre...,2004-04-01,True,Dolar AS kembali jatuh terhadap Euro setelah a...
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,https://finance.detik.com/moneter/d-122084/dit...,2004-04-01,True,Direktorat Jenderal (Ditjen) Pajak berhasil me...
...,...,...,...,...,...
354827,Penggantian Wesel KRL,https://finance.detik.com/foto-bisnis/d-489878...,2020-02-14,True,PT KCI dan PT KAI Daerah Operasi 1 Jakarta men...
354828,Berburu Aneka Sepeda Bekas di Jalan Malabar Ba...,https://finance.detik.com/foto-bisnis/d-489901...,2020-02-14,True,Jalan Malabar di Kota Bandung terkenal sebagai...
354829,Benang Gelasan Asli Sumedang Tembus Pasar Eksp...,https://finance.detik.com/foto-bisnis/d-489915...,2020-02-14,True,"Sejak 1990-an, Kampung gelasan di Tanjungsari,..."
354830,Bank Mantap Bagikan Deviden Rp 136 Miliar,https://finance.detik.com/foto-bisnis/d-489965...,2020-02-14,True,Bank Mantap menggelar Rapat Umum Pemegang Saha...


In [427]:
detik_judul

,judul,sub_judul,kategori,timestamp_publish,link,tanggal
0,OPEC Putuskan Kurangi Produksi,hasil sidang opec di wina,berita ekonomi bisnis,1080770940,https://finance.detik.com/berita-ekonomi-bisni...,01-04-2004
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",NaN,bursa dan valas,1080787800,https://finance.detik.com/bursa-dan-valas/d-12...,01-04-2004
2,Rupiah Dibuka Menguat di Rp 8570,NaN,moneter,1080788280,https://finance.detik.com/moneter/d-122087/rup...,01-04-2004
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",NaN,moneter,1080791460,https://finance.detik.com/moneter/d-122086/gre...,01-04-2004
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,NaN,moneter,1080799020,https://finance.detik.com/moneter/d-122084/dit...,01-04-2004
...,...,...,...,...,...,...
340627,"UMKM Resah Produk Impor, Ketua MPR Minta Ada P...",NaN,berita ekonomi bisnis,1638892559,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021
340628,Siap-siap Tarif PBB Bakal Naik,infografis,infografis,1638892835,https://finance.detik.com/infografis/d-5845257...,07-12-2021
340629,Korban Erupsi Semeru Diusulkan Dapat Keringana...,NaN,berita ekonomi bisnis,1638893242,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021
340630,"Segera Diresmikan Jokowi, Menhub Cek Bandara T...",NaN,infrastruktur,1638895340,https://finance.detik.com/infrastruktur/d-5845...,07-12-2021


In [438]:
detik_judul["published_at"] = pd.to_datetime(detik_judul['timestamp_publish'], unit='s') + pd.DateOffset(hours=7)
detik_judul

,judul,sub_judul,kategori,timestamp_publish,link,tanggal,published_at
0,OPEC Putuskan Kurangi Produksi,hasil sidang opec di wina,berita ekonomi bisnis,1080770940,https://finance.detik.com/berita-ekonomi-bisni...,01-04-2004,2004-04-01 05:09:00
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",NaN,bursa dan valas,1080787800,https://finance.detik.com/bursa-dan-valas/d-12...,01-04-2004,2004-04-01 09:50:00
2,Rupiah Dibuka Menguat di Rp 8570,NaN,moneter,1080788280,https://finance.detik.com/moneter/d-122087/rup...,01-04-2004,2004-04-01 09:58:00
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",NaN,moneter,1080791460,https://finance.detik.com/moneter/d-122086/gre...,01-04-2004,2004-04-01 10:51:00
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,NaN,moneter,1080799020,https://finance.detik.com/moneter/d-122084/dit...,01-04-2004,2004-04-01 12:57:00
...,...,...,...,...,...,...,...
340627,"UMKM Resah Produk Impor, Ketua MPR Minta Ada P...",NaN,berita ekonomi bisnis,1638892559,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021,2021-12-07 22:55:59
340628,Siap-siap Tarif PBB Bakal Naik,infografis,infografis,1638892835,https://finance.detik.com/infografis/d-5845257...,07-12-2021,2021-12-07 23:00:35
340629,Korban Erupsi Semeru Diusulkan Dapat Keringana...,NaN,berita ekonomi bisnis,1638893242,https://finance.detik.com/berita-ekonomi-bisni...,07-12-2021,2021-12-07 23:07:22
340630,"Segera Diresmikan Jokowi, Menhub Cek Bandara T...",NaN,infrastruktur,1638895340,https://finance.detik.com/infrastruktur/d-5845...,07-12-2021,2021-12-07 23:42:20


In [445]:
ordered_colums = ["judul", "isi", "kategori", "published_at", "link"]
rename = {"judul" : "title", "kategori" : "category", "isi" : "content"}

data_detik = (detik_artikel
               .set_index("link")
               .join(detik_judul[["kategori", "published_at", "link"]].set_index("link"))
               .reset_index()
               .drop("is_sukses", axis=1)[ordered_colums]
               .rename(columns=rename))

data_detik['website'] = "detik finance"
data_detik

,title,content,category,published_at,link,website
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance
...,...,...,...,...,...,...
340627,Penggantian Wesel KRL,PT KCI dan PT KAI Daerah Operasi 1 Jakarta men...,foto bisnis,2020-02-14 09:13:50,https://finance.detik.com/foto-bisnis/d-489878...,detik finance
340628,Berburu Aneka Sepeda Bekas di Jalan Malabar Ba...,Jalan Malabar di Kota Bandung terkenal sebagai...,foto bisnis,2020-02-14 12:10:20,https://finance.detik.com/foto-bisnis/d-489901...,detik finance
340629,Benang Gelasan Asli Sumedang Tembus Pasar Eksp...,"Sejak 1990-an, Kampung gelasan di Tanjungsari,...",foto bisnis,2020-02-14 13:47:43,https://finance.detik.com/foto-bisnis/d-489915...,detik finance
340630,Bank Mantap Bagikan Deviden Rp 136 Miliar,Bank Mantap menggelar Rapat Umum Pemegang Saha...,foto bisnis,2020-02-14 14:00:26,https://finance.detik.com/foto-bisnis/d-489965...,detik finance


In [446]:
data_detik.to_json("data/fix_artikel_detik.json")

#### Satukan Seluruh Data

In [456]:
full_data = pd.concat([data_detik, data_bisnis])
full_data['published_at'] = full_data.published_at.astype(str)
full_data = full_data.sort_values("published_at").reset_index(drop=True)

full_data

,title,content,category,published_at,link,website
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance
...,...,...,...,...,...,...
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com
550495,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Emiten pelayaran PT Pelita Samudera Shipping T...,Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com


In [457]:
full_data.to_json("data/fix_full_data.json")

### Cari Artikel Saham

In [460]:
full_data = pd.read_json("data/fix_full_data.json")
full_data

,title,content,category,published_at,link,website
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance
...,...,...,...,...,...,...
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com
550495,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Emiten pelayaran PT Pelita Samudera Shipping T...,Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com


In [464]:
all_saham = pd.read_csv("data/daftar_saham.csv", index_col=0)
all_saham

,kode,nama_perusahaan,keyword
0,AALI,PT Astra Agro Lestari Tbk,astra agro lestari
1,ABBA,PT Mahaka Media Tbk,mahaka media
2,ABDA,PT Asuransi Bina Dana Arta Tbk,asuransi bina dana arta
3,ABMM,PT ABM Investama Tbk,abm investama
4,ACES,PT Ace Hardware Indonesia Tbk,ace hardware indonesia
...,...,...,...
755,YULE,PT Yulie Sekuritas Indonesia Tbk,yulie sekuritas indonesia
756,ZBRA,PT Zebra Nusantara Tbk,zebra nusantara
757,ZINC,PT Kapuas Prima Coal Tbk,kapuas prima coal
758,ZONE,PT Mega Perintis Tbk,mega perintis


In [ ]:
def find_artikel_saham(row):
    print(row.name)
    title = row['title']
    content = row['content']
    artikel_saham = []
    
    if content is not None:
        content_lower = content.lower()

        def contains_word(sentance, word):
            return (' ' + word + ' ') in (' ' + sentance + ' ')


        for index, saham in all_saham.iterrows():
            get_artikel = False
            
            kode_saham = saham['kode']
            nama_perusahaan = saham['nama_perusahaan']

            tmp = {"kode_saham" : kode_saham, "nama_perusahaan" : nama_perusahaan, "index_artikel" : row.name}

            if contains_word(content, f"({kode_saham})"):
                tmp['keyword'] = kode_saham
                artikel_saham.append(tmp)
                get_artikel = True
            elif contains_word(content, nama_perusahaan):
                tmp['keyword'] = nama_perusahaan
                artikel_saham.append(tmp)
                get_artikel = True
            else:
                keywords = [] if saham['keyword'] is None else str(saham['keyword']).split(';')

                for keyword in keywords:
                    if contains_word(content_lower, keyword):
                        tmp['keyword'] = keyword
                        artikel_saham.append(tmp)
                        get_artikel = True
                        break
            
            if (not get_artikel) and (title is not None):
                if contains_word(title.lower(), nama_perusahaan):
                    tmp['keyword'] = "title"
                    artikel_saham.append(tmp)
                
    row['related_to_saham'] = artikel_saham
    return row

In [516]:
full_data2 = full_data.apply(find_artikel_saham, axis=1)
full_data2

546347
546348
546349
546350
546351
546352
546353
546354
546355
546356
546357
546358
546359
546360
546361
546362
546363
546364
546365
546366
546367
546368
546369
546370
546371
546372
546373
546374
546375
546376
546377
546378
546379
546380
546381
546382
546383
546384
546385
546386
546387
546388
546389
546390
546391
546392
546393
546394
546395
546396
546397
546398
546399
546400
546401
546402
546403
546404
546405
546406
546407
546408
546409
546410
546411
546412
546413
546414
546415
546416
546417
546418
546419
546420
546421
546422
546423
546424
546425
546426
546427
546428
546429
546430
546431
546432
546433
546434
546435
546436
546437
546438
546439
546440
546441
546442
546443
546444
546445
546446
546447
546448
546449
546450
546451
546452
546453
546454
546455
546456
546457
546458
546459
546460
546461
546462
546463
546464
546465
546466
546467
546468
546469
546470
546471
546472
546473
546474
546475
546476
546477
546478
546479
546480
546481
546482
546483
546484
546485
546486
546487
546488
546489

547519
547520
547521
547522
547523
547524
547525
547526
547527
547528
547529
547530
547531
547532
547533
547534
547535
547536
547537
547538
547539
547540
547541
547542
547543
547544
547545
547546
547547
547548
547549
547550
547551
547552
547553
547554
547555
547556
547557
547558
547559
547560
547561
547562
547563
547564
547565
547566
547567
547568
547569
547570
547571
547572
547573
547574
547575
547576
547577
547578
547579
547580
547581
547582
547583
547584
547585
547586
547587
547588
547589
547590
547591
547592
547593
547594
547595
547596
547597
547598
547599
547600
547601
547602
547603
547604
547605
547606
547607
547608
547609
547610
547611
547612
547613
547614
547615
547616
547617
547618
547619
547620
547621
547622
547623
547624
547625
547626
547627
547628
547629
547630
547631
547632
547633
547634
547635
547636
547637
547638
547639
547640
547641
547642
547643
547644
547645
547646
547647
547648
547649
547650
547651
547652
547653
547654
547655
547656
547657
547658
547659
547660
547661

548690
548691
548692
548693
548694
548695
548696
548697
548698
548699
548700
548701
548702
548703
548704
548705
548706
548707
548708
548709
548710
548711
548712
548713
548714
548715
548716
548717
548718
548719
548720
548721
548722
548723
548724
548725
548726
548727
548728
548729
548730
548731
548732
548733
548734
548735
548736
548737
548738
548739
548740
548741
548742
548743
548744
548745
548746
548747
548748
548749
548750
548751
548752
548753
548754
548755
548756
548757
548758
548759
548760
548761
548762
548763
548764
548765
548766
548767
548768
548769
548770
548771
548772
548773
548774
548775
548776
548777
548778
548779
548780
548781
548782
548783
548784
548785
548786
548787
548788
548789
548790
548791
548792
548793
548794
548795
548796
548797
548798
548799
548800
548801
548802
548803
548804
548805
548806
548807
548808
548809
548810
548811
548812
548813
548814
548815
548816
548817
548818
548819
548820
548821
548822
548823
548824
548825
548826
548827
548828
548829
548830
548831
548832

549862
549863
549864
549865
549866
549867
549868
549869
549870
549871
549872
549873
549874
549875
549876
549877
549878
549879
549880
549881
549882
549883
549884
549885
549886
549887
549888
549889
549890
549891
549892
549893
549894
549895
549896
549897
549898
549899
549900
549901
549902
549903
549904
549905
549906
549907
549908
549909
549910
549911
549912
549913
549914
549915
549916
549917
549918
549919
549920
549921
549922
549923
549924
549925
549926
549927
549928
549929
549930
549931
549932
549933
549934
549935
549936
549937
549938
549939
549940
549941
549942
549943
549944
549945
549946
549947
549948
549949
549950
549951
549952
549953
549954
549955
549956
549957
549958
549959
549960
549961
549962
549963
549964
549965
549966
549967
549968
549969
549970
549971
549972
549973
549974
549975
549976
549977
549978
549979
549980
549981
549982
549983
549984
549985
549986
549987
549988
549989
549990
549991
549992
549993
549994
549995
549996
549997
549998
549999
550000
550001
550002
550003
550004

,title,content,category,published_at,link,website,related_to_saham
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance,[]
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,[]
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance,[]
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance,[]
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance,[]
...,...,...,...,...,...,...,...
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'GIAA', 'nama_perusahaan': 'PT..."
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'PANS', 'nama_perusahaan': 'PT..."
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'CTRA', 'nama_perusahaan': 'PT..."
550495,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Emiten pelayaran PT Pelita Samudera Shipping T...,Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'PSSI', 'nama_perusahaan': 'PT..."


In [517]:
full_data2.to_json("data/fix_full_data2.json")

In [520]:
related_to_saham = []
for index, row in full_data2.iterrows():
    related_to_saham = related_to_saham + row['related_to_saham']
    
related_to_saham = pd.DataFrame(related_to_saham)
related_to_saham

,kode_saham,nama_perusahaan,index_artikel,keyword
0,INCO,PT Vale Indonesia Tbk,6,INCO
1,INCO,PT Vale Indonesia Tbk,7,INCO
2,INCO,PT Vale Indonesia Tbk,8,INCO
3,INCO,PT Vale Indonesia Tbk,9,INCO
4,BBCA,PT Bank Central Asia Tbk,12,BBCA
...,...,...,...,...
289284,PANS,PT Panin Sekuritas Tbk,550493,panin sekuritas
289285,CTRA,PT Ciputra Development Tbk,550494,ciputra development
289286,MIKA,PT Mitra Keluarga Karyasehat Tbk,550494,mitra keluarga karyasehat
289287,SMGR,PT Semen Indonesia,550494,PT Semen Indonesia


In [618]:
# simpan seluruh data artikel yang berhubungan dengan saham
article_related_to_saham = related_to_saham.index_artikel.unique()
article_related_to_saham = full_data2.iloc[article_related_to_saham, :]
article_related_to_saham.to_json("article_related_to_saham.json")
article_related_to_saham

,title,content,category,published_at,link,website,related_to_saham
6,Laba Bersih INCO 2003 Melonjak 245 Persen,Tingginya permintaan nikel membuat kinerja keu...,bursa dan valas,2004-04-01 14:27:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'kode_saham': 'INCO', 'nama_perusahaan': 'PT..."
7,Pemegang Saham INCO Setuju Pembagian Dividen 2003,Rapat Umum Pemegang Saham (RUPS) PT Internatio...,bursa dan valas,2004-04-01 14:56:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'kode_saham': 'INCO', 'nama_perusahaan': 'PT..."
8,"Beroperasi Akhir 2004, Petea Diharap Tambah Pr...",PT International Nickel Indonesia Tbk (INCO) m...,bursa dan valas,2004-04-01 14:57:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'kode_saham': 'INCO', 'nama_perusahaan': 'PT..."
9,INCO Akan Ajukan Pemerintah dan DPR ke Arbitrase,PT International Nickel Indonesia Tbk (INCO) a...,berita ekonomi bisnis,2004-04-01 15:08:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance,"[{'kode_saham': 'INCO', 'nama_perusahaan': 'PT..."
12,"IHSG Hanya Naik 1,976 Poin","Pada penutupan perdagangan hari ini, Kamis (1/...",bursa dan valas,2004-04-01 16:22:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'kode_saham': 'BBCA', 'nama_perusahaan': 'PT..."
...,...,...,...,...,...,...,...
550491,"Lakukan Restrukturisasi dan Terdampak PPKM, In...",Grup emiten peritel yang menaungi tiga brand y...,Korporasi,2021-12-09 20:20:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'HERO', 'nama_perusahaan': 'PT..."
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'GIAA', 'nama_perusahaan': 'PT..."
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'PANS', 'nama_perusahaan': 'PT..."
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'CTRA', 'nama_perusahaan': 'PT..."


In [530]:
related_to_saham.to_csv("data/related_to_saham.csv")

In [527]:
related_to_saham.nama_perusahaan.value_counts()

PT Bank Mandiri                          19481
PT Bank Rakyat Indonesia                 15500
PT Bank Central Asia Tbk                 10934
PT Bank Negara Indonesia                 10929
PT Aneka Tambang Tbk                      7141
                                         ...  
Indo Pureco Pratama Tbk                      6
PT Formosa Ingredient Factory Tbk            6
OBM Drilchem Tbk                             5
PT Bintang Samudera Mandiri Lines Tbk        1
Wahana Inti Makmur Tbk                       1
Name: nama_perusahaan, Length: 760, dtype: int64

In [531]:
related_to_saham.nama_perusahaan.value_counts().to_csv("data/related_to_saham_summary.csv")

Ternyata yang paling banyak adalah artikel yang relate dengan perbankan. Wajar sih ya yang discraping kan berita di detik finance dan bisnis market & finance. Oleh karena itu akan fokus aja ke 5 saham perbankan dengan jumlah artikel terbanyak

In [557]:
selected_artikel_bank = ["PT Bank Mandiri", "PT Bank Rakyat Indonesia", 
                 "PT Bank Central Asia Tbk", "PT Bank Negara Indonesia"]

selected_artikel_bank = related_to_saham[related_to_saham.nama_perusahaan.isin(selected_artikel_bank)]
selected_artikel_bank

,kode_saham,nama_perusahaan,index_artikel,keyword
4,BBCA,PT Bank Central Asia Tbk,12,BBCA
6,BBRI,PT Bank Rakyat Indonesia,12,BBRI
17,BMRI,PT Bank Mandiri,25,bank mandiri
25,BBCA,PT Bank Central Asia Tbk,40,BBCA
26,BBRI,PT Bank Rakyat Indonesia,40,BBRI
...,...,...,...,...
289251,BBRI,PT Bank Rakyat Indonesia,550460,BBRI
289252,BMRI,PT Bank Mandiri,550460,BMRI
289262,BBRI,PT Bank Rakyat Indonesia,550467,PT Bank Rakyat Indonesia
289275,BBCA,PT Bank Central Asia Tbk,550481,BBCA


In [558]:
selected_artikel_bank.index_artikel.unique().shape

(35888,)

In [560]:
selected_artikel = full_data.iloc[selected_artikel_bank.index_artikel.unique(), :]
selected_artikel = selected_artikel.reset_index().rename(columns={"index" : "index_artikel"})
selected_artikel

,index_artikel,title,content,category,published_at,link,website
0,12,"IHSG Hanya Naik 1,976 Poin","Pada penutupan perdagangan hari ini, Kamis (1/...",bursa dan valas,2004-04-01 16:22:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
1,25,"Pelaku Pasar Optimis Pemilu Lancar, IHSG Naik ...",Indeks harga saham gabungan pada penutupan Jum...,bursa dan valas,2004-04-02 16:36:09,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,40,"IHSG Melejit 13,710 Poin","Pada penutupan sesi pertama, Selasa (6/4/2004)...",bursa dan valas,2004-04-06 12:18:40,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
3,49,"IHSG Melonjak 20,896 Poin",Indeks Harga Saham Gabungan (IHSG) pada penutu...,bursa dan valas,2004-04-06 16:23:18,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
4,78,"Jelang Libur Panjang, Indeks Naik 5,218 Poin","Menjelang libur panjang, Indeks Harga Saham Ga...",bursa dan valas,2004-04-08 16:21:19,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
...,...,...,...,...,...,...,...
35883,550445,IFSoc: Tren Akuisisi Bank Kecil oleh Big Tech ...,Indonesia Fintech Society (IFSoc) melihat feno...,Perbankan,2021-12-09 14:36:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35884,550460,"Kekhawatiran Omicron Mereda Topang IHSG, Asing...",Indeks Harga Saham Gabungan (IHSG) ditutup nai...,Bursa & Saham,2021-12-09 15:44:00,https://market.bisnis.com/read/20211209/7/1475...,bisnis.com
35885,550467,UMKM EXPO(RT) BRILIANPRENEUR 2021: Mengusung U...,Usaha Mikro Kecil dan Menengah (UMKM) terus me...,Perbankan,2021-12-09 16:51:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35886,550481,BCA Pastikan Kantor Cabang di Lumajang Tetap B...,PT Bank Central Asia Tbk. (BBCA) menyatakan ka...,Perbankan,2021-12-09 18:50:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com


In [572]:
selected_bank = (selected_artikel_bank[['kode_saham', "nama_perusahaan"]]
                 .drop_duplicates()
                 .reset_index(drop=True)
                 .reset_index()
                 .rename(columns={"kode_saham" : "code", "nama_perusahaan" : "company_name", "index" : "id"}))
selected_bank

,id,code,company_name
0,0,BBCA,PT Bank Central Asia Tbk
1,1,BBRI,PT Bank Rakyat Indonesia
2,2,BMRI,PT Bank Mandiri


In [575]:
selected_artikel = selected_artikel.reset_index().rename(columns={"index" : "id"})
selected_artikel

,id,index_artikel,title,content,category,published_at,link,website
0,0,12,"IHSG Hanya Naik 1,976 Poin","Pada penutupan perdagangan hari ini, Kamis (1/...",bursa dan valas,2004-04-01 16:22:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
1,1,25,"Pelaku Pasar Optimis Pemilu Lancar, IHSG Naik ...",Indeks harga saham gabungan pada penutupan Jum...,bursa dan valas,2004-04-02 16:36:09,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,2,40,"IHSG Melejit 13,710 Poin","Pada penutupan sesi pertama, Selasa (6/4/2004)...",bursa dan valas,2004-04-06 12:18:40,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
3,3,49,"IHSG Melonjak 20,896 Poin",Indeks Harga Saham Gabungan (IHSG) pada penutu...,bursa dan valas,2004-04-06 16:23:18,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
4,4,78,"Jelang Libur Panjang, Indeks Naik 5,218 Poin","Menjelang libur panjang, Indeks Harga Saham Ga...",bursa dan valas,2004-04-08 16:21:19,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
...,...,...,...,...,...,...,...,...
35883,35883,550445,IFSoc: Tren Akuisisi Bank Kecil oleh Big Tech ...,Indonesia Fintech Society (IFSoc) melihat feno...,Perbankan,2021-12-09 14:36:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35884,35884,550460,"Kekhawatiran Omicron Mereda Topang IHSG, Asing...",Indeks Harga Saham Gabungan (IHSG) ditutup nai...,Bursa & Saham,2021-12-09 15:44:00,https://market.bisnis.com/read/20211209/7/1475...,bisnis.com
35885,35885,550467,UMKM EXPO(RT) BRILIANPRENEUR 2021: Mengusung U...,Usaha Mikro Kecil dan Menengah (UMKM) terus me...,Perbankan,2021-12-09 16:51:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35886,35886,550481,BCA Pastikan Kantor Cabang di Lumajang Tetap B...,PT Bank Central Asia Tbk. (BBCA) menyatakan ka...,Perbankan,2021-12-09 18:50:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com


Antar tabel masih dihubungkan dengan `code` saham dan `index_artikel`. Akan diatur lebih baik lagi sehingga untuk tabel artikel ada kolom `id` dan tabel saham juga ada kolom `id`-nya. Sehingga `id` tersebut nanti yang akan digunakan pada tabel intermediate, yaitu tabel `artikel_saham`

In [609]:
table_article_saham = (selected_artikel_bank
                         .rename(columns={"kode_saham" : "code"})
                         .set_index("code")
                         .join(selected_bank[['id', "code"]]
                               .set_index('code'))
                         .rename(columns={"id" : "saham_id"})
                         .set_index("index_artikel")
                         .join(selected_artikel[["id", "index_artikel"]]
                               .set_index("index_artikel"))
                         .rename(columns={"id" : "article_id"})
                         .drop(["nama_perusahaan"], axis=1)
                         .reset_index(drop=True))

table_article_saham.to_json("table_article_saham.json")
table_article_saham

,keyword,saham_id,article_id
0,BBCA,0,0
1,BBRI,1,0
2,bank mandiri,2,1
3,BBCA,0,2
4,BBRI,1,2
...,...,...,...
45910,BBRI,1,35884
45911,BMRI,2,35884
45912,PT Bank Rakyat Indonesia,1,35885
45913,BBCA,0,35886


In [610]:
table_articles = selected_artikel.drop(["id", "index_artikel"], axis=1)
table_articles.to_json("table_articles.json")
table_articles

,title,content,category,published_at,link,website
0,"IHSG Hanya Naik 1,976 Poin","Pada penutupan perdagangan hari ini, Kamis (1/...",bursa dan valas,2004-04-01 16:22:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
1,"Pelaku Pasar Optimis Pemilu Lancar, IHSG Naik ...",Indeks harga saham gabungan pada penutupan Jum...,bursa dan valas,2004-04-02 16:36:09,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,"IHSG Melejit 13,710 Poin","Pada penutupan sesi pertama, Selasa (6/4/2004)...",bursa dan valas,2004-04-06 12:18:40,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
3,"IHSG Melonjak 20,896 Poin",Indeks Harga Saham Gabungan (IHSG) pada penutu...,bursa dan valas,2004-04-06 16:23:18,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
4,"Jelang Libur Panjang, Indeks Naik 5,218 Poin","Menjelang libur panjang, Indeks Harga Saham Ga...",bursa dan valas,2004-04-08 16:21:19,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
...,...,...,...,...,...,...
35883,IFSoc: Tren Akuisisi Bank Kecil oleh Big Tech ...,Indonesia Fintech Society (IFSoc) melihat feno...,Perbankan,2021-12-09 14:36:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35884,"Kekhawatiran Omicron Mereda Topang IHSG, Asing...",Indeks Harga Saham Gabungan (IHSG) ditutup nai...,Bursa & Saham,2021-12-09 15:44:00,https://market.bisnis.com/read/20211209/7/1475...,bisnis.com
35885,UMKM EXPO(RT) BRILIANPRENEUR 2021: Mengusung U...,Usaha Mikro Kecil dan Menengah (UMKM) terus me...,Perbankan,2021-12-09 16:51:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com
35886,BCA Pastikan Kantor Cabang di Lumajang Tetap B...,PT Bank Central Asia Tbk. (BBCA) menyatakan ka...,Perbankan,2021-12-09 18:50:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com


In [611]:
table_saham = selected_bank.drop("id", axis=1)
table_saham.to_json("table_saham.json")
table_saham

,code,company_name
0,BBCA,PT Bank Central Asia Tbk
1,BBRI,PT Bank Rakyat Indonesia
2,BMRI,PT Bank Mandiri


## Ubah Format JSON
Beberapa perubahan dari sisi perubahan nama tabel dll

In [55]:
article_related_to_saham = pd.read_json("article_related_to_saham.json")

In [ ]:
def ubah_format(row):
    related_to_saham = row['related_to_saham']
    new_format = []
    
    for saham in related_to_saham:
        new_format.append({"code" : saham['kode_saham'], "company_name" : saham['nama_perusahaan'], 
                           "link" : row['link'], "keyword" : saham['keyword']})
    row['related_to_saham'] = new_format
    
    return row

article_related_to_saham = article_related_to_saham.apply(ubah_format, axis=1)
article_related_to_saham

In [56]:
related_to_saham = []
for index, row in article_related_to_saham.iterrows():
    related_to_saham = related_to_saham + row['related_to_saham']
    
related_to_saham = pd.DataFrame(related_to_saham)
related_to_saham

,code,company_name,link,keyword
0,INCO,PT Vale Indonesia Tbk,https://finance.detik.com/bursa-dan-valas/d-12...,INCO
1,INCO,PT Vale Indonesia Tbk,https://finance.detik.com/bursa-dan-valas/d-12...,INCO
2,INCO,PT Vale Indonesia Tbk,https://finance.detik.com/bursa-dan-valas/d-12...,INCO
3,INCO,PT Vale Indonesia Tbk,https://finance.detik.com/berita-ekonomi-bisni...,INCO
4,BBCA,PT Bank Central Asia Tbk,https://finance.detik.com/bursa-dan-valas/d-12...,BBCA
...,...,...,...,...
267351,PANS,PT Panin Sekuritas Tbk,https://market.bisnis.com/read/20211209/189/14...,panin sekuritas
267352,CTRA,PT Ciputra Development Tbk,https://market.bisnis.com/read/20211209/189/14...,ciputra development
267353,MIKA,PT Mitra Keluarga Karyasehat Tbk,https://market.bisnis.com/read/20211209/189/14...,mitra keluarga karyasehat
267354,SMGR,PT Semen Indonesia,https://market.bisnis.com/read/20211209/189/14...,PT Semen Indonesia


In [57]:
selected_bank = ["PT Bank Mandiri", "PT Bank Rakyat Indonesia", 
                 "PT Bank Central Asia Tbk", "PT Bank Negara Indonesia"]

selected_artikel_bank = related_to_saham[related_to_saham.company_name.isin(selected_bank)]
table_saham = (selected_artikel_bank[['code', "company_name"]]
                 .drop_duplicates()
                 .reset_index(drop=True)
                 .reset_index()
                 .rename(columns={"kode_saham" : "code", "nama_perusahaan" : "company_name", "index" : "id"}))

table_saham['id'] = table_saham['id'] + 1
table_saham

,id,code,company_name
0,1,BBCA,PT Bank Central Asia Tbk
1,2,BBNI,PT Bank Negara Indonesia
2,3,BBRI,PT Bank Rakyat Indonesia
3,4,BMRI,PT Bank Mandiri


In [58]:
table_article = (article_related_to_saham
                    .loc[article_related_to_saham.link.isin(selected_artikel_bank.link.to_list()), :]
                    .reset_index(drop=True)
                    .reset_index()
                    .rename(columns={"index" : "id"}))

table_article['id'] = table_article['id'] + 1
table_article

,id,title,content,category,published_at,link,website,related_to_saham
0,1,"IHSG Hanya Naik 1,976 Poin","Pada penutupan perdagangan hari ini, Kamis (1/...",bursa dan valas,2004-04-01 16:22:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'code': 'BBCA', 'company_name': 'PT Bank Cen..."
1,2,"Pelaku Pasar Optimis Pemilu Lancar, IHSG Naik ...",Indeks harga saham gabungan pada penutupan Jum...,bursa dan valas,2004-04-02 16:36:09,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'code': 'BMRI', 'company_name': 'PT Bank Man..."
2,3,Rupiah Alami Penguatan Terbatas,Pelaksanaan Pemilu yang berjalan lancar dan am...,moneter,2004-04-06 09:27:43,https://finance.detik.com/moneter/d-126488/rup...,detik finance,"[{'code': 'BBNI', 'company_name': 'PT Bank Neg..."
3,4,"IHSG Melejit 13,710 Poin","Pada penutupan sesi pertama, Selasa (6/4/2004)...",bursa dan valas,2004-04-06 12:18:40,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'code': 'AALI', 'company_name': 'PT Astra Ag..."
4,5,"IHSG Melonjak 20,896 Poin",Indeks Harga Saham Gabungan (IHSG) pada penutu...,bursa dan valas,2004-04-06 16:23:18,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,"[{'code': 'BMRI', 'company_name': 'PT Bank Man..."
...,...,...,...,...,...,...,...,...
33269,33270,IFSoc: Tren Akuisisi Bank Kecil oleh Big Tech ...,Indonesia Fintech Society (IFSoc) melihat feno...,Perbankan,2021-12-09 14:36:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com,"[{'code': 'AGRO', 'company_name': 'PT Bank Rak..."
33270,33271,"Kekhawatiran Omicron Mereda Topang IHSG, Asing...",Indeks Harga Saham Gabungan (IHSG) ditutup nai...,Bursa & Saham,2021-12-09 15:44:00,https://market.bisnis.com/read/20211209/7/1475...,bisnis.com,"[{'code': 'ASII', 'company_name': 'PT Astra In..."
33271,33272,UMKM EXPO(RT) BRILIANPRENEUR 2021: Mengusung U...,Usaha Mikro Kecil dan Menengah (UMKM) terus me...,Perbankan,2021-12-09 16:51:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com,"[{'code': 'BBRI', 'company_name': 'PT Bank Rak..."
33272,33273,BCA Pastikan Kantor Cabang di Lumajang Tetap B...,PT Bank Central Asia Tbk. (BBCA) menyatakan ka...,Perbankan,2021-12-09 18:50:00,https://finansial.bisnis.com/read/20211209/90/...,bisnis.com,"[{'code': 'BBCA', 'company_name': 'PT Bank Cen..."


In [60]:
table_article_saham = (selected_artikel_bank
                         .set_index("code")
                         .join(table_saham[['id', "code"]]
                               .set_index('code'))
                         .rename(columns={"id" : "saham_id"})
                         .set_index("link")
                         .join(table_article[["id", "link"]]
                               .set_index("link"))
                         .rename(columns={"id" : "article_id"})
                         .drop(["company_name"], axis=1)
                         .reset_index(drop=True))

table_article_saham

,keyword,saham_id,article_id
0,bank mandiri,4,2314
1,PT Bank Mandiri,4,2327
2,bank mandiri,4,2342
3,BMRI,4,2345
4,bank mandiri,4,2348
...,...,...,...
46116,PT Bank Negara Indonesia,2,33265
46117,BBCA,1,33268
46118,BBCA,1,33271
46119,BBRI,3,33271


In [63]:
article_related_to_saham.to_json("article_related_to_saham.json")
table_saham.to_json("table_saham.json")
table_article.to_json("table_articles.json")
table_article_saham.to_json("table_article_saham.json")

Simpan juga data yang tidak berhubungan dengan saham, udah capek capek dikumpulin masa dibuang

In [17]:
full_data2 = pd.read_json('data/fix_full_data2.json')
full_data2

,title,content,category,published_at,link,website,related_to_saham
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance,[]
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,[]
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance,[]
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance,[]
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance,[]
...,...,...,...,...,...,...,...
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'GIAA', 'nama_perusahaan': 'PT..."
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'PANS', 'nama_perusahaan': 'PT..."
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'CTRA', 'nama_perusahaan': 'PT..."
550495,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Emiten pelayaran PT Pelita Samudera Shipping T...,Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'PSSI', 'nama_perusahaan': 'PT..."


In [18]:
full_data2['related_to_saham'] = full_data2.related_to_saham.apply(lambda saham : None if len(saham) == 0 else saham)
full_data2

,title,content,category,published_at,link,website,related_to_saham
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance,None
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance,None
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance,None
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance,None
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance,None
...,...,...,...,...,...,...,...
550492,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Pengadilan Negeri Jakarta Pusat mengabulkan tu...,Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'GIAA', 'nama_perusahaan': 'PT..."
550493,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Kinerja indeks saham syariah Jakarta Islamic I...,Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'PANS', 'nama_perusahaan': 'PT..."
550494,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Artha Sekuritas Indonesia memprediksi Indeks H...,Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,bisnis.com,"[{'kode_saham': 'CTRA', 'nama_perusahaan': 'PT..."
550495,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Emiten pelayaran PT Pelita Samudera Shipping T...,Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com,"[{'kode_saham': 'PSSI', 'nama_perusahaan': 'PT..."


In [22]:
(full_data2[full_data2.related_to_saham.isnull()]
 .reset_index(drop=True).drop("related_to_saham", axis=1).to_json("article_norelated_to_saham.json"))

In [64]:
a = pd.read_json("article_norelated_to_saham.json")
a

,title,content,category,published_at,link,website
0,OPEC Putuskan Kurangi Produksi,"Meskipun harga minyak saat ini melonjak, OPEC ...",berita ekonomi bisnis,2004-04-01 05:09:00,https://finance.detik.com/berita-ekonomi-bisni...,detik finance
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",Indeks Harga Saham Gabungan pada Kamis (1/4/20...,bursa dan valas,2004-04-01 09:50:00,https://finance.detik.com/bursa-dan-valas/d-12...,detik finance
2,Rupiah Dibuka Menguat di Rp 8570,"Meskipun secara perlahan, nilai tukar Rupiah p...",moneter,2004-04-01 09:58:00,https://finance.detik.com/moneter/d-122087/rup...,detik finance
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",Dolar AS kembali jatuh terhadap Euro setelah a...,moneter,2004-04-01 10:51:00,https://finance.detik.com/moneter/d-122086/gre...,detik finance
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,Direktorat Jenderal (Ditjen) Pajak berhasil me...,moneter,2004-04-01 12:57:00,https://finance.detik.com/moneter/d-122084/dit...,detik finance
...,...,...,...,...,...,...
409473,"Booming Kripto, Aplikasi Trading Pintu Miliki ...","Platform jual beli dan investasi aset kripto, ...",Komoditas,2021-12-09 18:39:00,https://market.bisnis.com/read/20211209/94/147...,bisnis.com
409474,Restrukturisasi Utang Pan Brothers PBRX Dikabu...,None,Korporasi,2021-12-09 19:09:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com
409475,Fintech Wealth-Tech Harus Ikut Jaga Gairah Inv...,Indonesia Fintech Society (IFSoc) melihat pera...,Fintech,2021-12-09 19:10:00,https://finansial.bisnis.com/read/20211209/563...,bisnis.com
409476,"Ekspansi Crazy Rich Jahja Tear Tjahjana, Kuasa...",None,Korporasi,2021-12-09 20:12:00,https://market.bisnis.com/read/20211209/192/14...,bisnis.com
